# Analysis of Treasury Bill Pricing and Interest Rate Risk

## Introduction
Treasury bills, or T-bills, are debt instruments with short-term maturity periods ranging from a few days to 52 weeks. T-bills are zero-coupon fixed-income investments, i.e., no coupon payments are paid during their term. Instead, treasury bills are priced so that the bill holder receives the par value $V_{P}$ at the end of the term. A zero-coupon T-bill with an annual market interest rate of $\bar{r}$, specified at the time of purchase, and a T-year term has the fair price:

$$
V_{B} = \frac{V_{P}}{\left(1+\bar{r}\right)^{T}}
$$

The fair price $V_{B}$ of the treasury bill is the future par value $V_{P}$ of the T-bill discounted to today by the market interest rate $\bar{r}$. Treasury bills are issued for specific periods, such as 4, 8, 13, 26, and 52 weeks.

## Scenario
You purchased a 52-week T-bill with a par value $V_{P}=1000$ USD and interest rate $\bar{r}$. However, four weeks (28-days) after purchasing the T-bill, the interest rate is now $\bar{r}^{\prime}$. Should you `sell` the T-bill before it matures or `hold` the T-bill to term?

### Tasks 
 1. Load, analyze and visualize historical T-bill interest rate data sets from 2019 to 2023 computed daily by [Federal Reserve Bank of New York](https://www.newyorkfed.org). Store these data in a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) where the year, e.g., `"2022"` are the keys and the data sets of the values. 
 1. Develop a strategy to decide between a `hold` or `sell` action for your T-bill.
 1. Backtest your stratetgy using historical interest rate data for 52-week T-bills
     1. Implement your strategy and compute the number of sucess and failures of strategy

### Setup
In the following code block we setup the computational aspects of the problem by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this excercise and custom types to model the components of our problem.

#### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.

#### Functions
`Include.jl` loads the [Julia functions](https://docs.julialang.org/en/v1/manual/functions/):

* The `loadratesfile(year::String = "2022") -> DataFrame` function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) encoding the year in the `YYYY` format, e.g., `"2019"` or `"2020"` and returns a [DataFrame](https://dataframes.juliadata.org/stable/) holding the daily interest rates for US Treasury Bills for various durations found on [Treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates). This project has data from `"2019"`, `"2020"`, `"2021"`, `"2022"` and `"2022"`. The year parameter has a default value of `"2022"`.
* The `build(model::Type{MyUSTreasuryBillModel}, data::NamedTuple) -> MyUSTreasuryBillModel` function takes information in the `data` arguement (the par value, the rate and the duration of the T-bill) and returns an instance of the `MyUSTreasuryBillModel` custom type.

#### Types
`Include.jl` loads some [problem specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryBillModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$ and the interest rate $\bar{r}$ for a zero-coupon treasury bill. You construct a `MyUSTreasuryBillModel` instance using the `build` method described above.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-1`


## Task 1: Load and analyze historical interest rate data
The rates are based on recently auctioned bills with various maturities and collected by the Federal Reserve Bank of New York every business day. The Bank Discount rate is based on par value and discount amount, while the Coupon Equivalent is based on purchase price and discount, and can be used to compare yields.

### Load the historical rate data into a dictionary

In [2]:
rate_data_dictionary = Dict{String,DataFrame}();
years = ["2019","2020","2021","2022","2023"];
for year ∈ years
    rate_data_dictionary[year] = loadratesfile(year=year);
end

### Construct a discrete lattice model of future interest rates
To create a discrete lattice model for future interest rates, we need to gather data on three parameters: $p$, $u$, and $d$:

* The $p$ parameter denotes the probability that an interest rate will rise, i.e., an `up` move between two time periods $j\rightarrow{j+1}$. A binary lattice model only allow `up` and `down` moves, thus, the probability of a `down` move is $1-p$.
* The $u$ parameter denotes the amount of an `up` move, i.e., if $r_{j}$ denotes the interest rate in period $j$, and $r_{j+1}$ denotes the interest rate in the next period, then an `up` move gives $r_{j+1} = u\cdot{r}_{j}$. 
* The $d$ parameter denotes the amount of a `down` move, i.e., if $r_{j}$ denotes the interest rate in period $j$, and $r_{j+1}$ denotes the interest rate in the next period, then a `down` moves gives $r_{j+1} = d\cdot{r}_{j}$.

## Task 2: Develop a Strategy for the Hold versus Sell action